<a href="https://colab.research.google.com/github/nazzang49/dacon/blob/main/dacon_nli_competition_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dacon NLI Competition Practice**
https://dacon.io/competitions/official/235875/overview/description
- Strarified K-Fold
- Soft Voting Ensemble on Probs
- Model Architecture Transformation
- HPO

In [48]:
# !pip install transformers datasets

## **Import Modules**

In [49]:
import pandas as pd
import os
import numpy as np
import torch
import torch.optim as optim
import random

from torch.utils.data import Dataset, DataLoader
from torch import nn
from tqdm import tqdm, tqdm_notebook
from glob import glob
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, get_cosine_with_hard_restarts_schedule_with_warmup

## **Fix Seed**

In [50]:
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## **Load Data**
- DataFrame Head
- DataFrame Describe
- Pandas Profiling

In [51]:
train = pd.read_csv("/content/train_data.csv")
test = pd.read_csv("/content/test_data.csv")
submission = pd.read_csv("/content/sample_submission.csv")

In [52]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


In [73]:
test.head()

,index,premise,hypothesis,label,premise_,hypothesis_,text_sum
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,answer,다만 조금 좁아서 케리어를 펼치기 불편합니다.[SEP],케리어를 펼치기에 공간이 충분했습니다.,다만 조금 좁아서 케리어를 펼치기 불편합니다.[SEP] 케리어를 펼치기에 공간이 충...
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,answer,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요[SEP],시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요[SEP] 시먼역보다는 샤오난먼역에 ...
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,answer,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.[SEP],무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.[SEP] 무엇인가 말을...
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,answer,몇 번을 다시봐도 볼 때마다 가슴이 저민다.[SEP],다시 봤을때는 무덤덤했다.,몇 번을 다시봐도 볼 때마다 가슴이 저민다.[SEP] 다시 봤을때는 무덤덤했다.
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,answer,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.[SEP]",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.[S..."


In [53]:
train.describe()

,index
count,24998.000000
mean,12498.500000
std,7216.445351
min,0.000000
25%,6249.250000
50%,12498.500000
75%,18747.750000
max,24997.000000


In [70]:
test.describe()

,index
count,1666.00000
mean,832.50000
std,481.07709
min,0.00000
25%,416.25000
50%,832.50000
75%,1248.75000
max,1665.00000


In [54]:
device = torch.device("cuda:0")

## **Load Model**
- Load Config
- Load Model
- Load Tokenizer

In [55]:
config = AutoConfig.from_pretrained('klue/bert-base', num_labels=3)
bertmodel = AutoModelForSequenceClassification.from_config(config)
tok = AutoTokenizer.from_pretrained('klue/bert-base')

In [ ]:
bertmodel

In [ ]:
set_seeds(42)

## **Tokenization + Preprocess**
- Create Input Format
- Convert Labels into Digits
- Stratified K-Fold on Dataset

In [57]:
label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}

In [71]:
train["premise_"] = train["premise"] + "[SEP]"
train["hypothesis_"] = train["hypothesis"]

test["premise_"] = test["premise"] + "[SEP]"
test["hypothesis_"] = test["hypothesis"]

train["text_sum"] = train.premise_ + " " + train.hypothesis_
test["text_sum"] = test.premise_ + " " + test.hypothesis_

train_content = []
test_content = []

for i, text in enumerate(train.text_sum):
    train_content.append(list([text, label_dict[train.label[i]]]))
    
for i, text in enumerate(test.text_sum):
    test_content.append([text])
    
dataset_train = train_content
dataset_test = test_content

In [63]:
class BERTDataset(Dataset):
    def __init__(self, tokenized_examples, labels=None):
        self.tokenized_examples = tokenized_examples
        self.labels = labels

    def __getitem__(self, i):
        item = {key: value[i] for key, value in self.tokenized_examples.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[i])
        return item
    
    def __len__(self):
        return len(self.tokenized_examples['input_ids'])

In [72]:
# train
dataset_train_sentences = [row[0] for row in dataset_train]
dataset_train_labels = [row[1] for row in dataset_train]

# test
dataset_test_sentences = [row[0] for row in dataset_test]

In [60]:
tokenized_train_sentences = tok(
    dataset_train_sentences,    # premise + hypothesis
    add_special_tokens=True,
    return_token_type_ids=True, # roberta = False
    padding=True,
    truncation=True,
    max_length=100,
    return_tensors="pt"
)

tokenized_test_sentences = tok(
    dataset_test_sentences,     # premise + hypothesis
    add_special_tokens=True,
    return_token_type_ids=True, # roberta = False
    padding=True,
    truncation=True,
    max_length=100,
    return_tensors="pt"
)

In [61]:
tokenized_train_sentences['input_ids'][0]

tensor([    2, 14441,  2073, 12382, 13169,  2200,  3797, 21505,  9005,  2259,
         3997,  2031,  2079,  3661, 31221,  5845,  2200,  2112,    16,  5950,
        15351, 17788,  7285,   748,  2088, 22048,  2470,  1132, 21893, 15351,
         6481, 27135,  5417,  4084,  1972,  2145, 17524,  2138, 15526,  2259,
          575, 28674,    18,     3, 14441,  2079,  3883,  2031,  2079,  5845,
        28674,    18,     3,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [62]:
len(tokenized_train_sentences['input_ids'])

20000

In [40]:
# dataset
data_train = BERTDataset(tokenized_train_sentences, dataset_train_labels)
data_test = BERTDataset(tokenized_test_sentences)

In [90]:
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import StratifiedKFold
from datasets import load_metric, load_dataset, concatenate_datasets
from pandas.core.frame import DataFrame

In [74]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [111]:
df_data_train = DataFrame.from_records(data_train) # dict-based BERTDataset to DataFrame

In [114]:
for n_fold, (train_idx, val_idx) in enumerate(skf.split(data_train, data_train.labels)):
    df_data_train.loc[val_idx, 'fold'] = n_fold # iloc fail

In [116]:
df_data_train.head()

,input_ids,token_type_ids,attention_mask,labels,fold
0,"[tensor(2), tensor(14441), tensor(2073), tenso...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",tensor(1),1.0
1,"[tensor(2), tensor(3840), tensor(2073), tensor...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",tensor(1),4.0
2,"[tensor(2), tensor(8345), tensor(3627), tensor...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",tensor(0),0.0
3,"[tensor(2), tensor(4104), tensor(16955), tenso...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",tensor(2),2.0
4,"[tensor(2), tensor(4739), tensor(4297), tensor...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",tensor(2),4.0


## **Trainer**
- Create TrainingArguments
- Create Trainer
- Do Train

In [117]:
batch_size = 32

In [ ]:
# optimizer = AdamwHF
TrainingArguments(
    run_name='dacon-klue-nli',
    output_dir='klue-nli',
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy='steps',
    evaluation_strategy='steps',
    save_steps=200,
    eval_steps=200,
    debug='underflow_overflow',
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='get_cosine_with_hard_restarts_schedule_with_warmup',
    warmup_steps=150,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy', # validation accuracy
    report_to='all',
    # gradient_checkpointing=True     # save memory but, slower backward
)

In [68]:
# metric for evaluation
metric = load_metric('glue', 'qnli')

In [69]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
import gc
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

## **Inference**
- Convert Digits to Labels
- Submission

In [ ]:
# inference
result = []
model.eval()
with torch.no_grad():
    for batch_id, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = batch['input_ids'].to(device)
        segment_ids = batch['attention_mask'].to(device)
        out = model(token_ids, segment_ids)
        result.append(out.logits)

In [ ]:
result_ = []
for i in result:
    for j in i:
        result_.append(int(torch.argmax(j)))
        
out = [list(label_dict.keys())[_] for _ in result_]
submission["label"] = out
submission.to_csv("sample_submission.csv", index = False)